In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import re

import geopandas as gpd
import requests,json


from shapely import geos
from shapely.geometry import Point
import fiona
from fiona.crs import from_epsg,from_string

from geopandas import GeoDataFrame
from shapely.geometry import Polygon
from geopandas import GeoSeries


from shapely import geometry
from shapely.geometry import shape,LineString,Point,Polygon

import osmnx as ox
import networkx as nx

import folium

from folium import IFrame

import seaborn as sns
import branca

In [3]:
def rest(address):
    try:
        url = 'https://api.tomtom.com/search/2/geocode/{}.json?&key=h2n93GznJsnsDHtv2vPpdT9GmJijp9o9&countrySet=TWN&language=zh-TW&limit=1'.format(address)
        response = requests.get(url)
        data = response.text
        js = json.loads(str(data))
        return js['results'][0]['position']
    except Exception as e:
        print(f'An Error occurred: {e}')
        return {'lat':0,'lon':0}
    
def trans(gdf):
    gdf.crs = {'init' :'epsg:3824'} # 避免資料沒設，這邊再重新給一次
    gdf=gdf.to_crs({'init' :'epsg:4326'})
    return gdf

def utf2asc(s):
    return str(str(s).encode('ascii', 'xmlcharrefreplace'))[2:-1]

heading3 = """<h3>{}</h3>""".format

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import os
os.chdir(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data')

# 村里邊界資料整理

In [6]:
gdf=gpd.read_file(r'C:/Users/RT009223/GIS/村里/VILLAGE_MOI_1090423.shp',encoding='utf-8')
gdf=gdf.loc[:,['COUNTYNAME','TOWNNAME','VILLNAME','geometry']]
gdf=trans(gdf)


In [7]:
gdf=gdf.dropna().reset_index(drop=True)

In [8]:
gdf.COUNTYNAME=list(gdf.COUNTYNAME.apply(lambda x:x.replace('臺','台')))

In [9]:
print(gdf.shape)
gdf.columns=['縣市','行政區','村里','geometry']
gdf.head()


(7760, 4)


,縣市,行政區,村里,geometry
0,高雄市,林園區,中門里,"POLYGON ((120.36772 22.49564, 120.36427 22.498..."
1,高雄市,林園區,港埔里,"POLYGON ((120.37325 22.49123, 120.37147 22.492..."
2,高雄市,旗津區,上竹里,"POLYGON ((120.28976 22.57317, 120.28577 22.577..."
3,高雄市,旗津區,中興里,"POLYGON ((120.30320 22.55568, 120.30284 22.557..."
4,連江縣,南竿鄉,津沙村,"POLYGON ((119.91684 26.14109, 119.91683 26.141..."


In [10]:
#gdf[gdf['村里']=='明德里'].area

In [11]:
#gdf[gdf['村里']=='明德里'].to_crs({'init': 'epsg:3857'}).area/ 10**6

In [12]:
#gdf[gdf['村里']=='明德里'].to_crs({'init': 'epsg:3395'}).area/ 10**6

# 會員資料整理

In [13]:
df_get_back=pd.read_table(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\HOLA會員位置(無正確地址的資料)_一般卡_2020H1.txt',sep=',')

In [14]:
df_get_back=df_get_back[-df_get_back['村里'].isna()].reset_index(drop=True)

In [15]:
df_get_back.head()

,id,縣市,行政區,村里
0,20100340242510,新北市,新莊區,昌隆里
1,20131213000053,桃園市,平鎮區,湧安里
2,20110715000289,新北市,五股區,更寮里
3,20190324000981,彰化縣,田中鎮,復興里
4,19960323000068,台北市,萬華區,富福里


# Mapping資料

In [16]:
#geom=[Point(data.lon,data.lat) for idx, data in df_get_back.iterrows()]

In [17]:
#crs = {'init': 'epsg:4326'}
#df_get_back = gpd.GeoDataFrame(df_get_back, crs=crs, geometry=geom)

In [18]:
#df_get_back.shape

In [19]:
#df_get_back = gpd.tools.sjoin(df_get_back,gdf, how="inner", op='intersects')
#df_get_back.head()

In [20]:
#df_get_back.columns=['id','addr','zone','lon','lat','geometry','index_right','縣市','行政區','村里']
#df_get_back=df_get_back.loc[:,['id','addr','lon','lat','縣市','行政區','村里']]
#df_get_back.head()

In [21]:
#result.columns=['id','addr','lon','lat','geometry','index_right','縣市','行政區','村里']
#result=result.copy().reset_index(drop=True)

In [22]:
#result2=result.copy()

In [23]:
#result=result.iloc[:,1:]
#result.head()

In [24]:
#result.to_csv('TLW會員位置(經整理_配對村里)_一般卡.txt',index=0)

In [25]:
result=pd.read_table(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\HOLA會員位置(經整理_配對村里)_一般卡_2020H1.txt',sep=',')
print(result.shape)
result.head()

(777658, 7)


,id,addr,lon,lat,縣市,行政區,村里
0,19991001001467,嘉義市西區永和里6鄰力行街60號 ...,120.438100,23.474250,嘉義市,西區,永和里
1,19980516000744,高雄市新興區正氣里12鄰青年一路200巷3之4號 ...,120.308807,22.622596,高雄市,新興區,正氣里
2,20060304000601,嘉義市西區劉厝里2鄰大聖三街16號 ...,120.419144,23.461140,嘉義市,西區,劉厝里
3,20041126000005,嘉義市東區林森里12鄰長榮街112號 ...,120.454170,23.484390,嘉義市,東區,林森里
4,20001122000482,嘉義縣民雄鄉中央村10鄰田中央63號 ...,120.403315,23.533626,嘉義縣,民雄鄉,中央村


In [26]:
df_get_back=df_get_back[-df_get_back.id.isin(result.id)]

In [27]:
result=pd.concat([result,df_get_back]).reset_index(drop=True)

In [28]:
result.shape

(788792, 7)

# 會員消費額

In [29]:
sales=pd.read_csv(r'C:\Users\RT009223\TestriteGroup\滲透率專案\01_raw data\HOLA2020H1一般卡會員消費金額.csv',error_bad_lines=False,header=None)
sales.columns=['id','sale']
print(sales.shape)
print(sales.sale.sum())
print(sales.id.isin(result.id).sum())

(918039, 2)
3792643313.399999
788792


In [30]:
result=pd.merge(result,sales).reset_index(drop=True)
result.shape

(788792, 8)

# 人口資料整理

In [31]:
pop=pd.read_excel(r'台灣202006年村里人口.xlsx')

In [32]:
pop['縣市']=pop['縣市'].str.replace('\xa0','')
pop['行政區']=pop['行政區'].str.replace('\xa0','')
pop['村里']=pop['村里'].str.replace('\xa0','')

In [33]:

#公舘里~公館里
#上舘里~上館里
#石[曹]里~石𥕢里
#鷄林里~雞林里
#水磜村~水祭村
#珊湖里~珊瑚里

In [34]:
gdf.columns=['縣市','行政區','村里','geometry']

In [35]:
gdf['縣市']=list(gdf['縣市'].apply(lambda x:x.replace('臺','台')))
gdf['行政區']=list(gdf['行政區'].apply(lambda x:x.replace('臺','台')))
gdf['村里']=list(gdf['村里'].apply(lambda x:x.replace('臺','台')))
pop['縣市']=list(pop['縣市'].apply(lambda x:x.replace('臺','台')))
pop['行政區']=list(pop['行政區'].apply(lambda x:x.replace('臺','台')))
pop['村里']=list(pop['村里'].apply(lambda x:x.replace('臺','台')))

gdf['村里']=gdf['村里'].apply(lambda x:x.replace('舘','館'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('舘','館'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('双','雙'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('双','雙'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
gdf['村里']=gdf['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))

gdf['村里']=gdf['村里'].apply(lambda x:x.replace('欍','瓊'))

pop['村里']=pop['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))

pop['村里']=pop['村里'].apply(lambda x:x.replace('欍','瓊'))


pop['村里']=pop['村里'].apply(lambda x:x.replace('溝垻里','溝埧里'))
pop['村里']=pop['村里'].apply(lambda x:x.replace('崙峰里','崙峯里'))

In [36]:
result['村里']=result['村里'].apply(lambda x:x.replace('舘','館'))
result['村里']=result['村里'].apply(lambda x:x.replace('石[曹]里','石𥕢里'))
result['村里']=result['村里'].apply(lambda x:x.replace('双','雙'))
result['村里']=result['村里'].apply(lambda x:x.replace('鷄林里','雞林里'))
result['村里']=result['村里'].apply(lambda x:x.replace('水磜村','水祭村'))
result['村里']=result['村里'].apply(lambda x:x.replace('珊湖里','珊瑚里'))
result['村里']=result['村里'].apply(lambda x:x.replace('欍','瓊'))

In [37]:
#result2=result.groupby(['縣市','行政區','村里'])['id'].count().reset_index()
result2=result.groupby(['縣市','行政區','村里']).agg({'id':'count', 'sale': 'sum'}).reset_index()
result2.columns=['縣市','行政區','村里','會員數','消費額']
result2.shape

(7524, 5)

In [38]:
result2['平均消費額']=round(result2['消費額']/result2['會員數'])

In [39]:
result2=pd.merge(pop,result2,how='left')
result2.shape

(7760, 8)

In [40]:
result2[-(result2['縣市']+result2['行政區']+result2['村里']).isin(pop['縣市']+pop['行政區']+pop['村里'])]['縣市'].unique()

array([], dtype=object)

In [41]:
pop[-(pop['行政區']+pop['村里']).isin(gdf['行政區']+gdf['村里'])]

,縣市,行政區,村里,戶口,人口


In [42]:
#gdf[(gdf['行政區']+gdf['村里']).apply(lambda x:x=='龍崎區崎頂里')]

In [43]:
#pop[pop['行政區']=='龍崎區']['村里'].unique()

In [44]:
#pop[pop['村里']=='崎頂里']

In [45]:
#gdf[gdf['村里']=='崎頂里']

In [46]:
#gdf[gdf['行政區']=='新化區']['村里'].unique()

In [47]:
result2['會員數'].sum()

788792.0

In [48]:
gdf_final=pd.merge(gdf,result2,how='right')

In [49]:
gdf_final['會員數'][gdf_final['會員數'].isna()]=0

In [50]:
gdf_final['site']=gdf_final['縣市']+gdf_final['行政區']+gdf_final['村里']
gdf_final

,縣市,行政區,村里,geometry,戶口,人口,會員數,消費額,平均消費額,site
0,高雄市,林園區,中門里,"POLYGON ((120.36772 22.49564, 120.36427 22.498...",1124,2884,48.0,127872.00,2664.0,高雄市林園區中門里
1,高雄市,林園區,港埔里,"POLYGON ((120.37325 22.49123, 120.37147 22.492...",822,2054,29.0,46825.00,1615.0,高雄市林園區港埔里
2,高雄市,旗津區,上竹里,"POLYGON ((120.28976 22.57317, 120.28577 22.577...",1386,3228,46.0,79963.09,1738.0,高雄市旗津區上竹里
3,高雄市,旗津區,中興里,"POLYGON ((120.30320 22.55568, 120.30284 22.557...",1069,2886,55.0,92555.00,1683.0,高雄市旗津區中興里
4,連江縣,南竿鄉,津沙村,"POLYGON ((119.91684 26.14109, 119.91683 26.141...",111,441,0.0,NaN,NaN,連江縣南竿鄉津沙村
...,...,...,...,...,...,...,...,...,...,...
7755,基隆市,七堵區,實踐里,"POLYGON ((121.70123 25.08284, 121.70167 25.081...",1683,4434,73.0,495159.89,6783.0,基隆市七堵區實踐里
7756,基隆市,七堵區,堵北里,"MULTIPOLYGON (((121.69626 25.08189, 121.69423 ...",1616,4256,93.0,631453.54,6790.0,基隆市七堵區堵北里
7757,基隆市,七堵區,堵南里,"POLYGON ((121.69469 25.08739, 121.69373 25.086...",934,2357,52.0,385324.75,7410.0,基隆市七堵區堵南里
7758,基隆市,暖暖區,暖東里,"POLYGON ((121.74707 25.09037, 121.74748 25.090...",193,459,8.0,153825.00,19228.0,基隆市暖暖區暖東里


In [51]:
gdf_final['ratio']=gdf_final['會員數']/gdf_final['戶口']
gdf_final['ratio2']=gdf_final['ratio']
gdf_final.loc[gdf_final['ratio2']>0.4,'ratio2']=0.4
gdf_final['percent']=gdf_final['ratio'].map(lambda x:str(round(x*100,1))+'%')
#gdf_final=gdf_final.set_index(gdf_final.site)
gdf_final['MarketPotential']=(gdf_final['戶口']-gdf_final['會員數']).astype('int')

In [52]:
gdf_final['平均消費額'].describe()

count      7524.000000
mean       3766.151648
std        2951.821267
min          33.000000
25%        2568.500000
50%        3488.500000
75%        4428.000000
max      140399.000000
Name: 平均消費額, dtype: float64

In [53]:
gdf_final['平均消費額2']=gdf_final['平均消費額']
gdf_final['平均消費額2'][gdf_final['平均消費額']>10000]=10000

In [54]:
#plt.boxplot(gdf_final.MarketPotential)

In [55]:
#gdf_final.sort_values(by='ratio',ascending=False).head(10)

In [56]:
#(result.addr.str.contains('保新里')&result.addr.str.contains('汐止區')).sum()

In [57]:
#result[result['村里']=='保新里'].head()

In [58]:
#gdf_final.sort_values(by=['ratio','戶口'],ascending=[True,False]).head(10)

In [59]:
gdf_final['MarketPotential_normalization']=(gdf_final['MarketPotential']-gdf_final['MarketPotential'].min())/(gdf_final['MarketPotential'].max()-gdf_final['MarketPotential'].min())
gdf_final['MarketPotential_density']=(gdf_final['MarketPotential'])/(gdf_final.to_crs({'init': 'epsg:3395'}).area/ 10**6)
gdf_final['MarketPotential_density_normalization']=(gdf_final['MarketPotential_density']-gdf_final['MarketPotential_density'].min())/(gdf_final['MarketPotential_density'].max()-gdf_final['MarketPotential_density'].min())
gdf_final['percent']=gdf_final['ratio'].map(lambda x:str(round(x*100,1))+'%')

In [60]:
gdf_final['平均消費額_normalization']=(gdf_final['平均消費額2']-gdf_final['平均消費額2'].min())/(gdf_final['平均消費額2'].max()-gdf_final['平均消費額2'].min())
gdf_final.loc[gdf_final['平均消費額_normalization'].isna(),['平均消費額_normalization']]=0

In [61]:
gdf_final.head()

,縣市,行政區,村里,geometry,戶口,人口,會員數,消費額,平均消費額,site,ratio,ratio2,percent,MarketPotential,平均消費額2,MarketPotential_normalization,MarketPotential_density,MarketPotential_density_normalization,平均消費額_normalization
0,高雄市,林園區,中門里,"POLYGON ((120.36772 22.49564, 120.36427 22.498...",1124,2884,48.0,127872.00,2664.0,高雄市林園區中門里,0.042705,0.042705,4.3%,1076,2664.0,0.075899,437.472573,0.010811,0.263971
1,高雄市,林園區,港埔里,"POLYGON ((120.37325 22.49123, 120.37147 22.492...",822,2054,29.0,46825.00,1615.0,高雄市林園區港埔里,0.035280,0.035280,3.5%,793,1615.0,0.055540,518.673928,0.012818,0.158724
2,高雄市,旗津區,上竹里,"POLYGON ((120.28976 22.57317, 120.28577 22.577...",1386,3228,46.0,79963.09,1738.0,高雄市旗津區上竹里,0.033189,0.033189,3.3%,1340,1738.0,0.094892,1011.124622,0.024991,0.171065
3,高雄市,旗津區,中興里,"POLYGON ((120.30320 22.55568, 120.30284 22.557...",1069,2886,55.0,92555.00,1683.0,高雄市旗津區中興里,0.051450,0.051450,5.1%,1014,1683.0,0.071439,326.196119,0.008060,0.165546
4,連江縣,南竿鄉,津沙村,"POLYGON ((119.91684 26.14109, 119.91683 26.141...",111,441,0.0,NaN,NaN,連江縣南竿鄉津沙村,0.000000,0.000000,0.0%,111,NaN,0.006475,80.140670,0.001978,0.000000


In [62]:
gdf_final.to_excel(r'C:\Users\RT009223\TestriteGroup\滲透率專案\05_HOLA\result\HOLA2020H1台灣.xlsx',index=0)

# HOLA店點(部分)

In [2]:
HOLA=pd.read_excel('HOLA店點.xlsx')
HOLA

NameError: name 'pd' is not defined

# 畫圖

In [64]:
# 各車站定位
台北車站=[25.047741, 121.517384]
中壢車站=[24.953737, 121.225607]
台中車站=[24.137264, 120.686669]
嘉義車站=[23.479119, 120.441138]

#各縣市邊框顏色
print(gdf_final['縣市'].unique())
site=['基隆市','新北市', '台北市','桃園市','新竹縣','新竹市','苗栗縣','台中市','彰化縣','南投縣',
      '雲林縣','嘉義縣','嘉義市','台南市',
      '高雄市','屏東縣','宜蘭縣','花蓮縣','台東縣',
      '澎湖縣','金門縣','連江縣']

site_colors={'基隆市':'#F17C67','新北市':'#F17C67', '台北市':'#F17C67',
            '桃園市':'#33A6B8','新竹縣':'#33A6B8','新竹市':'#33A6B8','苗栗縣':'#33A6B8',
            '台中市':'#00896C','彰化縣':'#00896C','南投縣':'#00896C','澎湖縣':'#00896C','金門縣':'#00896C','連江縣':'#00896C',
            '雲林縣':'#465D4C','嘉義縣':'#465D4C','嘉義市':'#465D4C','台南市':'#465D4C',
            '高雄市':'#6F3381','屏東縣':'#6F3381',
            '宜蘭縣':'#F7D94C','花蓮縣':'#F7D94C','台東縣':'#F7D94C'}


#想畫顏色的變數
variable='ratio2'

#設定色階
high=0.4
low=0.0
p=high/8
print(p)

colorscale = branca.colormap.linear.YlOrRd_09.scale(low, high)
colorscale

gdf_final['id']=gdf_final.index
color=gdf_final.set_index('id')[variable]


['高雄市' '連江縣' '屏東縣' '花蓮縣' '新竹市' '台東縣' '台中市' '嘉義縣' '桃園市' '南投縣' '雲林縣' '新竹縣'
 '嘉義市' '台南市' '基隆市' '澎湖縣' '金門縣' '苗栗縣' '宜蘭縣' '彰化縣' '新北市' '台北市']
0.05


In [65]:
colorscale

In [66]:
# 顏色 style
def style_function(feature):
    colors = color.get(int(feature['id']), None)
    site_color=site_colors.get(feature['properties']['縣市'])
    return {
        'fillOpacity': 0.5,
        'weight': 1,
        'color':site_color,
        'fillColor': '#fff' if colors is None else colorscale(colors)
    }

In [67]:
# 設定圖例
colormap=colorscale.to_step(index=[0, p*1, p*2, p*3, p*4,p*5,p*6,p*7,p*8])
colormap.caption ='Member Penetration Rates'

In [68]:
mm=folium.Map(location=台北車站,zoom_start=14)
for s in site:
    folium.GeoJson(
        gdf_final[gdf_final['縣市']==s][['geometry','縣市','行政區','村里','戶口','人口','會員數','ratio','percent','MarketPotential','MarketPotential_density','平均消費額','平均消費額_normalization']],
        name=s, 
        style_function=style_function,
        highlight_function=lambda x: {'weight':3, 
                                        'color':site_colors.get(x['properties']['縣市']),
                                        'fillOpacity':0.75},
        smooth_factor=2.0,
        tooltip=folium.features.GeoJsonTooltip(fields=['縣市','行政區','村里','戶口','percent','平均消費額',],
                                                  aliases=['縣市','行政區','村里','戶數','滲透率','平均消費額'], 
                                                  labels=True, 
                                                  sticky=True,
                                                  toLocaleString=True
                                                 )
    ).add_to(mm)

for i in range(HOLA.shape[0]):
    if HOLA.type[i]==0:
        iframe = IFrame(html=heading3(utf2asc(HOLA.name[i])),width=250,height=50)
        popup = folium.Popup(iframe)
        folium.Marker(location=[HOLA.lat[i], HOLA.lon[i]],
                       popup=popup,
                       icon=folium.Icon(icon='info-sign',
                                        color='lightblue')).add_to(mm)
    elif HOLA.type[i]==2:
        iframe = IFrame(html=heading3(utf2asc(HOLA.name[i])),width=250,height=50)
        popup = folium.Popup(iframe)
        folium.Marker(location=[HOLA.lat[i], HOLA.lon[i]],
                       popup=popup,
                       icon=folium.Icon(icon='info-sign',
                                        color='beige')).add_to(mm)
    else:  
        iframe = IFrame(html=heading3(utf2asc(HOLA.name[i])),width=250,height=50)
        popup = folium.Popup(iframe)
        folium.Marker(location=[HOLA.lat[i], HOLA.lon[i]],
                       popup=popup,
                       icon=folium.Icon(icon='info-sign',
                                        color='gray')).add_to(mm)    

colormap.add_to(mm)

folium.LayerControl().add_to(mm)


In [72]:
#mm

In [69]:
#北北基 桃竹苗 中彰投 雲嘉南
mm.save(r'C:\Users\RT009223\TestriteGroup\滲透率專案\05_HOLA\result\台灣2020H1年HOLA(含Petite)滲透率.html')